In [ ]:
import numpy as np
from tqdm import tqdm
import PIL
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.xception import preprocess_input,Xception
import math
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.decomposition import PCA

## USING XCEPTION NETWORKS TO PREPROCESS CALTECH DATASET

In [ ]:
img_size =224

model = Xception(weights='imagenet', include_top=False,input_shape=(img_size, img_size, 3),pooling='max')

In [ ]:
batch_size = 64
root_dir ='../input/caltech256/'

img_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

datagen = img_gen.flow_from_directory(root_dir,
                                        target_size=(img_size, img_size),
                                        batch_size=batch_size,
                                        class_mode=None,
                                        shuffle=False)

num_images = len(datagen.filenames)
num_epochs = int(math.ceil(num_images / batch_size))

feature_list = model.predict_generator(datagen, num_epochs,verbose = 1)

In [ ]:
print("Total images   = ", len(datagen.classes))
print("Shape of feature_list = ", feature_list.shape)

In [ ]:
plt.imshow(plt.imread('../input/caltech256/256_ObjectCategories/008.bathtub/008_0002.jpg'))
plt.show()

In [ ]:

filenames = [root_dir + '/' + s for s in datagen.filenames]


## ALGORITHM USED KNN (n=6)

In [ ]:
neighbors = NearestNeighbors(n_neighbors=6,
                             algorithm='ball_tree',
                             metric='euclidean')
neighbors.fit(feature_list)

### CHOOSING AN IMAGE FROM FLOWER-COLOR-IMAGES

In [ ]:
img_path ='../input/flower-color-images/flower_images/flower_images/0011.png'
input_shape = (img_size, img_size, 3)
img = image.load_img(img_path, target_size=(input_shape[0], input_shape[1]))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
preprocessed_img

In [ ]:
plt.imshow(plt.imread('../input/flower-color-images/flower_images/flower_images/0011.png'))
plt.show()

### TEST CASE FOR THE DATASET CALTECH IS AN IMAGE FORM FLOWER DATASET

In [ ]:
test_img_features = model.predict(preprocessed_img, batch_size=1)

_, indices = neighbors.kneighbors(test_img_features)

### FINDING THE IMAGE ON THE BASIS OF SIMILIARITY OF IMAGE MATRIX ON THE DATASET

In [ ]:
def similar_images(indices):
    plt.figure(figsize=(15,10))
    plotnumber = 1    
    for index in indices:
        if plotnumber<=len(indices) :
            ax = plt.subplot(2,3,plotnumber)
            plt.imshow(mpimg.imread(filenames[index]), interpolation='lanczos') 
            plt.xlabel(img_path.split('.')[0] + 'Similar Image'+str(index)+'\n',fontsize=20)
            plotnumber+=1
    plt.tight_layout()

print(indices.shape)

plt.imshow(mpimg.imread(img_path), interpolation='lanczos')
plt.xlabel(img_path.split('.')[0] + '_Original Image',fontsize=20)
plt.show()
print('********* Predictions ***********')
print(len(indices[0]))
similar_images(indices[0])

## DIMENSIONALITY REDUCTION

In [ ]:
pca = PCA(n_components=120)
pca.fit(feature_list)
compressed_features = pca.transform(feature_list)

In [ ]:
neighbors_pca_features = NearestNeighbors(n_neighbors=6,
                             algorithm='brute',
                             metric='euclidean').fit(compressed_features)

In [ ]:
test_img_compressed = pca.transform(test_img_features)
distances, indices = neighbors_pca_features.kneighbors(test_img_compressed)
print(indices.shape)
plt.imshow(mpimg.imread(img_path), interpolation='lanczos')
plt.xlabel(img_path.split('.')[0] + '_Original Image',fontsize=20)
plt.show()
print('********* Predictions  after PCA ***********')
similar_images(indices[0])

In [ ]:
img_path ='../input/cat-and-dog/training_set/training_set/dogs/dog.1005.jpg'
input_shape = (img_size, img_size, 3)
img = image.load_img(img_path, target_size=(input_shape[0], input_shape[1]))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)

In [ ]:
plt.imshow(plt.imread('../input/cat-and-dog/training_set/training_set/dogs/dog.1005.jpg'))
plt.show()

In [ ]:
test_img_features = model.predict(preprocessed_img, batch_size=1)

_, indices = neighbors.kneighbors(test_img_features)

In [ ]:

plt.imshow(mpimg.imread(img_path), interpolation='lanczos')
plt.xlabel(img_path.split('.')[0] + '_Original Image',fontsize=20)
plt.show()
print('********* Predictions ***********')
print(len(indices[0]))
similar_images(indices[0])

In [ ]:
test_img_compressed = pca.transform(test_img_features)
distances, indices = neighbors_pca_features.kneighbors(test_img_compressed)
print(indices.shape)
plt.imshow(mpimg.imread(img_path), interpolation='lanczos')
plt.xlabel(img_path.split('.')[0] + '_Original Image',fontsize=20)
plt.show()
print('********* Predictions after PCA ***********')
similar_images(indices[0])